# Data Preparation

In [73]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
pd.options.mode.chained_assignment = None

root = 'C:/Data/instacart-market-basket-analysis/'

import xgboost as xgb

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import roc_auc_score, roc_curve, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE

In [2]:
orders = pd.read_csv(root + 'orders.csv', 
                 dtype={
                        'order_id': np.int32,
                        'user_id': np.int64,
                        'eval_set': 'category',
                        'order_number': np.int16,
                        'order_dow': np.int8,
                        'order_hour_of_day': np.int8,
                        'days_since_prior_order': np.float32})


order_products_train = pd.read_csv(root + 'order_products__train.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

order_products_prior = pd.read_csv(root + 'order_products__prior.csv', 
                                 dtype={
                                        'order_id': np.int32,
                                        'product_id': np.uint16,
                                        'add_to_cart_order': np.int16,
                                        'reordered': np.int8})

product_features = pd.read_pickle(root + 'product_features.pkl')

user_features = pd.read_pickle(root + 'user_features.pkl')

user_product_features = pd.read_pickle(root + 'user_product_features.pkl')

products = pd.read_csv(root +'products.csv')

aisles = pd.read_csv(root + 'aisles.csv')

departments = pd.read_csv(root + 'departments.csv')

In [13]:
df = orders.merge(order_products_train, on = 'order_id', how = 'inner')
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered
0,1187899,1,train,11,4,8,14.0,196,1,1
1,1187899,1,train,11,4,8,14.0,25133,2,1
2,1187899,1,train,11,4,8,14.0,38928,3,1
3,1187899,1,train,11,4,8,14.0,26405,4,1
4,1187899,1,train,11,4,8,14.0,39657,5,1


In [14]:
df.shape

(1384617, 10)

In [7]:
product_features.head()

,product_id,mean_add_to_cart_order,total_orders,total_reorders,reorder_percentage,unique_users,order_first_time_total_cnt,order_second_time_total_cnt,is_organic,second_time_percent,...,department_household,department_international,department_meat seafood,department_missing,department_other,department_pantry,department_personal care,department_pets,department_produce,department_snacks
0,1,5.801836,1852,1136.0,0.613391,716,716,276,0,0.385475,...,0,0,0,0,0,0,0,0,0,1
1,2,9.888889,90,12.0,0.133333,78,78,8,0,0.102564,...,0,0,0,0,0,1,0,0,0,0
2,3,6.415162,277,203.0,0.732852,74,74,36,0,0.486486,...,0,0,0,0,0,0,0,0,0,0
3,4,9.507599,329,147.0,0.446809,182,182,64,0,0.351648,...,0,0,0,0,0,0,0,0,0,0
4,5,6.466667,15,9.0,0.600000,6,6,4,0,0.666667,...,0,0,0,0,0,1,0,0,0,0


In [8]:
user_features.head()

,user_id,avg_dow,std_dow,avg_doh,std_doh,avg_since_order,std_since_order,total_orders_by_user,total_products_by_user,total_unique_product_by_user,total_reorders_by_user,reorder_propotion_by_user,average_order_size,reorder_in_order,orders_3,orders_2,orders_1,reorder_3,reorder_2,reorder_1
0,1,2.644068,1.256194,10.542373,3.500355,18.542374,10.559066,10,59,18,41.0,0.694915,5.900000,0.705833,6,6,9,0.666667,1.0,0.666667
1,2,2.005128,0.971222,10.441026,1.649854,14.902564,9.671712,14,195,102,93.0,0.476923,13.928571,0.447961,19,9,16,0.578947,0.0,0.625000
2,3,1.011364,1.245630,16.352273,1.454599,10.181818,5.867395,12,88,33,55.0,0.625000,7.333333,0.658817,6,5,6,0.833333,1.0,1.000000
3,4,4.722222,0.826442,13.111111,1.745208,11.944445,9.973330,5,18,17,1.0,0.055556,3.600000,0.028571,7,2,3,0.142857,0.0,0.000000
4,5,1.621622,1.276961,15.729730,2.588958,10.189189,7.600577,4,37,23,14.0,0.378378,9.250000,0.377778,9,5,12,0.444444,0.4,0.666667


In [10]:
user_product_features.head()

,user_id,product_id,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1,196,10,9,0.900000,1.400000,17.600000,10,1.0,1.0,1.0
1,1,10258,9,8,0.888889,3.333333,19.555555,10,1.0,1.0,1.0
2,1,10326,1,0,0.000000,5.000000,28.000000,5,0.0,0.0,0.0
3,1,12427,10,9,0.900000,3.300000,17.600000,10,1.0,1.0,1.0
4,1,13032,3,2,0.666667,6.333333,21.666666,10,1.0,0.0,0.0


In [15]:
df = df.merge(product_features, on = 'product_id', how = 'left')
df = df.merge(user_features, on = 'user_id', how = 'left')
df = df.merge(user_product_features, on = ['user_id', 'product_id'], how = 'left')
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,reorder_1,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1187899,1,train,11,4,8,14.0,196,1,1,...,0.666667,10.0,9.0,0.900,1.4,17.600,10.0,1.0,1.0,1.0
1,1187899,1,train,11,4,8,14.0,25133,2,1,...,0.666667,8.0,7.0,0.875,4.0,20.125,10.0,1.0,1.0,1.0
2,1187899,1,train,11,4,8,14.0,38928,3,1,...,0.666667,1.0,0.0,0.000,4.0,30.000,10.0,0.0,0.0,0.0
3,1187899,1,train,11,4,8,14.0,26405,4,1,...,0.666667,2.0,1.0,0.500,5.0,14.500,4.0,0.0,0.0,0.0
4,1187899,1,train,11,4,8,14.0,39657,5,1,...,0.666667,1.0,0.0,0.000,3.0,30.000,10.0,0.0,0.0,0.0


The dataframe has null values because the product was never bought earlier by a user

In [16]:
df.shape

(1384617, 212)

In [22]:
df.isnull().sum().sort_values(ascending = False)

is_reorder_1                        555793
avg_add_to_cart_by_user             555793
total_product_orders_by_user        555793
total_product_reorders_by_user      555793
user_product_reorder_percentage     555793
is_reorder_2                        555793
avg_days_since_last_bought          555793
is_reorder_3                        555793
last_ordered_in                     555793
aisle_instant foods                      9
aisle_doughs gelatins bake mixes         9
aisle_food storage                       9
aisle_first aid                          9
aisle_feminine care                      9
aisle_facial care                        9
aisle_eye ear care                       9
aisle_energy sports drinks               9
aisle_energy granola bars                9
aisle_eggs                               9
aisle_dry pasta                          9
aisle_dog food care                      9
aisle_fresh fruits                       9
aisle_dish detergents                    9
aisle_diges

Filling values with 0 which help model know that these products are bought by user first time.

In [72]:
null_cols = ['is_reorder_1', 'is_reorder_2', 'is_reorder_3', 'avg_add_to_cart_by_user', 'total_product_orders_by_user',
    'total_product_reorders_by_user', 'user_product_reorder_percentage', 
    'avg_days_since_last_bought', 'last_ordered_in']

df[null_cols] = df[null_cols].fillna(0)

In [37]:
null_vals_prodcuts = df[df['aisle_instant foods'].isnull()].product_id.to_list()

In [38]:
products[products.product_id.isin(null_vals_prodcuts)]

,product_id,product_name,aisle_id,department_id
3717,3718,Wasabi Cheddar Spreadable Cheese,21,16
25382,25383,Chocolate Go Bites,61,19
27498,27499,Non-Dairy Coconut Seven Layer Bar,100,21
36232,36233,Water With Electrolytes,100,21
37702,37703,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...,75,17
43724,43725,Sweetart Jelly Beans,100,21
45970,45971,12 Inch Taper Candle White,101,17
49539,49540,Pure Squeezed Lemonade,31,7


In [39]:
order_products_prior[order_products_prior.product_id.isin(null_vals_prodcuts)]

,order_id,product_id,add_to_cart_order,reordered


Seems like all these products were never bought by any users in the prior orders

In [41]:
null_products = products[products.product_id.isin(null_vals_prodcuts)].merge(aisles, on = 'aisle_id', how = 'inner').\
                                                        merge(departments, on = 'department_id', how = 'inner')
null_products

,product_id,product_name,aisle_id,department_id,aisle,department
0,3718,Wasabi Cheddar Spreadable Cheese,21,16,packaged cheese,dairy eggs
1,25383,Chocolate Go Bites,61,19,cookies cakes,snacks
2,27499,Non-Dairy Coconut Seven Layer Bar,100,21,missing,missing
3,36233,Water With Electrolytes,100,21,missing,missing
4,43725,Sweetart Jelly Beans,100,21,missing,missing
5,37703,Ultra Sun Blossom Liquid 90 loads Fabric Enhan...,75,17,laundry,household
6,45971,12 Inch Taper Candle White,101,17,air fresheners candles,household
7,49540,Pure Squeezed Lemonade,31,7,refrigerated,beverages


In [61]:
k = df.copy()

In [85]:
for i in range(null_products.shape[0]):
    aisle_name = str('aisle_' + null_products.aisle[i])
    
    if df.columns.isin([aisle_name]).any():
        df.loc[df.product_id == null_products.product_id[i], aisle_name] = 1
    
    department_name = str('department_' + null_products.department[i])
    
    if df.columns.isin([department_name]).any():
        df.loc[df.product_id == null_products.product_id[i], department_name] = 1

In [93]:
aisle_cols = [i for i in df.columns if 'aisle' in i]
department_cols = [i for i in df.columns if 'department' in i]

In [94]:
df[aisle_cols] = df[aisle_cols].fillna(0)
df[department_cols] = df[department_cols].fillna(0)

In [95]:
df.isnull().sum().sort_values(ascending = False)

order_second_time_total_cnt            9
mean_add_to_cart_order                 9
second_time_percent                    9
is_organic                             9
order_first_time_total_cnt             9
unique_users                           9
reorder_percentage                     9
total_reorders                         9
total_orders                           9
asile_reorder_percentage               9
aisle_doughs gelatins bake mixes       0
aisle_energy granola bars              0
aisle_energy sports drinks             0
aisle_eye ear care                     0
aisle_facial care                      0
aisle_feminine care                    0
aisle_first aid                        0
aisle_eggs                             0
aisle_food storage                     0
aisle_fresh dips tapenades             0
aisle_dry pasta                        0
is_reorder_1                           0
aisle_dog food care                    0
aisle_fresh herbs                      0
aisle_dish deter

In [101]:
null_cols2 = ['order_second_time_total_cnt', 'mean_add_to_cart_order', 'second_time_percent', 'is_organic',
              'order_first_time_total_cnt', 'unique_users', 'reorder_percentage', 'total_reorders', 'total_orders',
              'asile_reorder_percentage' ]
df[null_cols2].dtypes

order_second_time_total_cnt    float64
mean_add_to_cart_order         float64
second_time_percent            float64
is_organic                     float64
order_first_time_total_cnt     float64
unique_users                   float64
reorder_percentage             float64
total_reorders                 float64
total_orders                   float64
asile_reorder_percentage       float64
dtype: object

In [102]:
df[null_cols2] = df[null_cols2].fillna(0)

Saving final dataframe

In [103]:
df.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,reorder_1,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1187899,1,train,11,4,8,14.0,196,1,1,...,0.666667,10.0,9.0,0.900,1.4,17.600,10.0,1.0,1.0,1.0
1,1187899,1,train,11,4,8,14.0,25133,2,1,...,0.666667,8.0,7.0,0.875,4.0,20.125,10.0,1.0,1.0,1.0
2,1187899,1,train,11,4,8,14.0,38928,3,1,...,0.666667,1.0,0.0,0.000,4.0,30.000,10.0,0.0,0.0,0.0
3,1187899,1,train,11,4,8,14.0,26405,4,1,...,0.666667,2.0,1.0,0.500,5.0,14.500,4.0,0.0,0.0,0.0
4,1187899,1,train,11,4,8,14.0,39657,5,1,...,0.666667,1.0,0.0,0.000,3.0,30.000,10.0,0.0,0.0,0.0


In [104]:
df.isnull().any().any()

False

In [105]:
df.to_pickle(root + 'Finaldata.pkl')

In [106]:
df2 = pd.read_pickle(root +'Finaldata.pkl')
df2.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,reorder_1,total_product_orders_by_user,total_product_reorders_by_user,user_product_reorder_percentage,avg_add_to_cart_by_user,avg_days_since_last_bought,last_ordered_in,is_reorder_3,is_reorder_2,is_reorder_1
0,1187899,1,train,11,4,8,14.0,196,1,1,...,0.666667,10.0,9.0,0.900,1.4,17.600,10.0,1.0,1.0,1.0
1,1187899,1,train,11,4,8,14.0,25133,2,1,...,0.666667,8.0,7.0,0.875,4.0,20.125,10.0,1.0,1.0,1.0
2,1187899,1,train,11,4,8,14.0,38928,3,1,...,0.666667,1.0,0.0,0.000,4.0,30.000,10.0,0.0,0.0,0.0
3,1187899,1,train,11,4,8,14.0,26405,4,1,...,0.666667,2.0,1.0,0.500,5.0,14.500,4.0,0.0,0.0,0.0
4,1187899,1,train,11,4,8,14.0,39657,5,1,...,0.666667,1.0,0.0,0.000,3.0,30.000,10.0,0.0,0.0,0.0


Yayyyyy. Ready for some cool modeling now :p